In [1]:
#Import packages 
import math
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import pandas as pd
import seaborn as sns
sns.set_style("darkgrid")
import datetime as dt
import os
from tqdm import tqdm
import folium
import random
import warnings
warnings.filterwarnings('ignore')

import matplotlib as mpl
from folium import plugins
from folium.plugins import HeatMap
import json

import base64
from io import BytesIO
from folium import IFrame

from matplotlib import dates as mdates

In [2]:
#Create dataframe with different car types

#Link to the three vehicles
#https://ev-database.org/car/1106/Nissan-Leaf
#https://ev-database.org/car/1203/Volkswagen-ID3-Pro-S
#https://ev-database.org/car/1406/Tesla-Model-S-Plaidplus

d={'Model':['Nissan Leaf','Volkswagen ID.3 Pro S','Tesla Model S Plaid+'], 'Battery':[36,77,115], 'Range':[220,450,670], 'Efficency':[164,171,172], 'Charge Speed':[12,55,82], 'Link':['https://ev-database.org/car/1106/Nissan-Leaf','https://ev-database.org/car/1203/Volkswagen-ID3-Pro-S','https://ev-database.org/car/1406/Tesla-Model-S-Plaidplus']}
EVs=pd.DataFrame(d)

In [3]:
#Import dataset and remove dead chargers
df_all=pd.read_csv('Clean_Dundee.csv',sep=';',decimal=',')
df_all['End Timestamp']= pd.to_datetime(df_all['End Timestamp'], format="%Y-%m-%d %H.%M.%S")
df_all['Start Timestamp']= pd.to_datetime(df_all['Start Timestamp'], format="%Y-%m-%d %H.%M.%S")
df_all['Charge Time']=df_all['End Timestamp'] - df_all['Start Timestamp']

dead=['50262','50843','50232', '50270','50273','50373','50677','50840','50912','50913','50914','51085','51356','51241','51422','51423','51424','51425','51426','51427','51428','51429','51547','51548','51549','51550']
df_all=df_all.loc[~df_all['CP ID'].isin(dead)]
#df_all.loc[~df_all['CP ID'].isin(dead)]['CP ID'].unique()
df_all=df_all.reset_index(drop=True)
df_all['Type']=df_all['Type'].fillna('Unknown')

In [4]:
#Import predictions as an csv
df_predictions=pd.read_csv('PredictionsKDE.csv', index_col=0)
df_predictions.index=pd.to_datetime(df_predictions.index)
df_predictions.columns=[float(i) for i in df_predictions.columns.tolist()]

In [5]:
# Function that finds chargers near you
def find_chargers_near_you(r,lat,lon,df_all): 
    df=df_all.copy()
    # Radius of earth
    R = 6373.0
    
    # Convert EV chargers coordinates to radians 
    df['Latitude'] = [math.radians(i) for i in df['Latitude']]
    df['Longitude'] = [math.radians(i) for i in df['Longitude']]

    # Convert user position
    lat = math.radians(lat)
    lon = math.radians(lon)

    # Calculate the difference
    df['dlon'] =  [i-lon for i in df['Longitude']]
    df['dlat'] = [i-lat for i in df['Latitude']]

    # Calculate the distance
    df['a'] = [math.sin(df['dlat'][i] / 2)**2 + math.cos(lat) * math.cos(df['Latitude'][i]) * math.sin(df['dlon'][i] / 2)**2 for i in range(len(df))]
    df['c'] = [2 * math.atan2(math.sqrt(i), math.sqrt(1 - i)) for i in df['a']]
    df['distance'] = R * df['c']


    CPIDs=df[df['distance']<=r]['CP ID'].unique().tolist()
    CPIDs=[str(i) for i in CPIDs]
    
    return CPIDs

In [6]:
#lat=56.45900
#lon=-2.98232
#r = 0.5
#find_chargers_near_you(r,lat,lon,df_all)

In [7]:
# Function that generates map of Dundee
def generateBaseMap(default_location=[56.46679, -2.97028], default_zoom_start=12, zoom =False):
    
    if zoom:
        base_map = folium.Map(location=default_location, control_scale=True, zoom_start=default_zoom_start)
    
    else:
        base_map = folium.Map(location=default_location, control_scale=True, zoom_start=default_zoom_start,max_zoom=default_zoom_start,min_zoom=default_zoom_start)
    return base_map

In [8]:
#Function that generates different shades of green
def color(value):
    return mpl.colors.to_hex(plt.cm.Greens(value))

In [9]:
# Function that creates a heatmap 
def heatmap(df_predict,lat,lon,r,index_list):
    #Find chargers within your radius
    df_folium_map = df_all[df_all['CP ID'].isin(find_chargers_near_you(r,lat,lon,df_all))][["CP ID","Latitude","Longitude"]].groupby("CP ID").mean()
    
    #Prepare data for heat map, list of lists with lat,lon and heat to given time
    data_list = []
    df_temp_all = 1-(df_predict.groupby((df_predict.index.dayofweek) * 24 +  (df_predict.index.hour)).mean())

    for i in range(df_temp_all.shape[0]):
        df_temp = df_temp_all.T[i]
        df_temp = df_all.groupby('CP ID')[['Latitude','Longitude']].mean().merge(df_temp,right_index = True, left_index = True)

        data_list.append(df_temp.values.tolist())


    #Plot timeseries graf for your area
    chargers = df_folium_map.index.tolist()
    encoded = plot_weekly_avilability(1-df_predict,chargers,index_list)

    html = '<img src="data:image/png;base64,{}">'.format
    iframe = IFrame(html(encoded), width=700, height=400)
    popup = folium.Popup(iframe, max_width=1000)
    
    # create map heat map
    base_heatmaptime =generateBaseMap(zoom=True)

    gradient  = {0.4: color(0.4) , 0.65: color(0.65), 0.9: color(0.9)}

    hm = plugins.HeatMapWithTime(data = data_list, gradient=gradient, radius=31, auto_play=True,min_speed=9,name='availability heat').add_to(base_heatmaptime)
    folium.Marker(location=[lat, lon], popup=popup).add_to(base_heatmaptime)
    
    #Plot circle with user specified radius
    folium.Circle(location=[lat, lon],radius=r*1000).add_to(base_heatmaptime)
    
    #Plot marker for the EV chargers in your area
    for index,row in df_folium_map.iterrows():  
        latlon=df_folium_map.loc[index][["Latitude","Longitude"]]
        
        #Plot timeseries graf for each close by EV charger
        charger = index
        encoded = plot_weekly_avilability(1-df_predict,charger,index_list)
        
        html = '<img src="data:image/png;base64,{}">'.format
        iframe = IFrame(html(encoded), width=700, height=400)
        popup = folium.Popup(iframe, max_width=1000)
        
        #Plot marker with popup graph
        folium.CircleMarker(location=[latlon.iloc[0], latlon.iloc[1]],radius = 3,opacity=1,color='black', fill=True,fill_opacity=1, popup=popup).add_to(base_heatmaptime)
          
    #Add layers to map                      
    zoom = 12
    folium.TileLayer('cartodbpositron').add_to(base_heatmaptime)
    folium.TileLayer('stamentoner').add_to(base_heatmaptime)
    folium.LayerControl().add_to(base_heatmaptime)
    
    #Add title to plot
    title = 'HeatMap of EV Chargers in Dundee'
    title_html = '''
                 <h3 align="center" style="font-size:16px"><b>{}</b></h3>
                 '''.format(title)   
    base_heatmaptime.get_root().html.add_child(folium.Element(title_html))

    # Display the map
    return base_heatmaptime

In [10]:
# Function that plots the weekly availability
def plot_weekly_avilability(df_grid_pred,charger,index_list):


    weekdays = ["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"]
    hours = [i*6 for i in range(0,4)]*7
    
    #Mean data for each weekday
    df_availability=(df_grid_pred.groupby((df_grid_pred.index.dayofweek) * 24 +  (df_grid_pred.index.hour)).mean())

    fig, ax = plt.subplots(figsize = (10,5))
    
    #Average the charging availibility in an area if the chargers are a list
    if type(charger) == list:
        fig.suptitle('Average Weekly EV charger Availability in Your Area \n',fontweight='bold')
        ax.set(title='The red dots indicate when you are able to charge given your preferences. \nThe grey line is threshold for when you can be sure to find free connectors.', ylabel='% Available Chargers', xlabel='Time of Week')
        ax = (df_availability[charger].mean(axis=1)*100).plot();
        points = df_availability[charger].mean(axis=1)[index_list]*100
        plt.plot(points,'o',color='red')
    
    else:
        ax.set(title='Average Weekly EV charger Availability at this EV Charger', ylabel='% Available Chargers', xlabel='Time of Week')
        ax = df_availability[charger].plot();

    # Plot horizontal line at value 0.5
    ax.axhline(y=45, color='grey')
    # Adjust axis
    ax.set_ylim([0,100])
    ax.set_xlim([0,168])
    ax.set_xticks(range(0,168,24))
    ax.set_xticks(range(0,168,6),minor=True)
    ax.set_xticklabels(weekdays,rotation=90)
    ax.set_xticklabels(hours, minor=True)
    ax.grid(b=True, axis="x", which='major', color='#666666', linestyle='--', alpha=0.4)
    ax.grid(b=True, axis="y", which='major', color='#666666', linestyle='--', alpha=0.4)

    plt.close()
    
    #Rerturn picture as html string
    tmpfile = BytesIO()
    fig.savefig(tmpfile, format='png')
    encoded = base64.b64encode(tmpfile.getvalue()).decode('utf-8')

    return encoded


In [11]:
#lat=56.45900
#lon=-2.98232
#r = 0.5
#index_list = [31, 46, 47, 79, 120, 140, 142]

#heatmap(df_predictions,lat,lon,r,index_list)

In [12]:
# Import OR packages
from pulp import GLPK
from pulp import LpMaximize, LpProblem, LpStatus, lpSum, LpVariable,LpMinimize
from pulp import *

In [13]:
# Define the operation research model
def OR_BatterySize(X, O, H):
    model = LpProblem(name="Battery Size", sense=LpMinimize)

    # Define the decision variables
    K = {i: LpVariable(name=f"K{i}", lowBound=0) for i in range(1, 30)}
    S = {i: LpVariable(name=f"S{i}", lowBound=0) for i in range(1, 30)}
    B = LpVariable(name='B', lowBound=0)

    K[0]=0 # Constraint 6.2
        
    for i in range(1,30):
        # Add constraints
        model += (B+sum([K[j] for j in range(1,i)])-sum([X[j] for j in range(1,i)])==S[i], "Constraint_6.0_{}".format(i))
        model += (X[i] <=S[i], "Constraint_6.3_{}".format(i))
        model += (S[i]<=B, "Constraint_6.4_{}".format(i))
        model += (K[i] <= O[i]*H, "Constraint_6.5_{}".format(i))
        model += (K[i] <=B-S[i], "Constraint_6.6_{}".format(i))
        model += (50<=S[i], "Constraint_6.7_{}".format(i))
        

    # Set the objective
    model += B

    # Solve the optimization problem
    status = model.solve()

    # Get the results
    #print(f"status: {model.status}, {LpStatus[model.status]}")
    #print(f"objective: {model.objective.value()}")
    
    #for name, constraint in model.constraints.items():
        #print(f"{name}: {constraint.value()}")
        
    #info = [{' name':name, 'dual value':c.pi, 'slack': c.slack} for name, c in model.constraints.items()]
    infoK = [{' name':var.name, 'value':var.value()} for num,var in enumerate(K.values()) if num!=29]
    infoS = [{' name':var.name, 'value':var.value()} for num,var in enumerate(S.values()) if num!=29]
    return model.objective.value(), infoK, infoS
    #return info

In [14]:
#X=[44]*7*5
#X.insert(0,0)
#O=[12,0,0,20,0,0,12]*5
#O.insert(0,0)
#H=50
#obj, infoK,infoS=OR_BatterySize(X, O, H)
#info=OR_BatterySize(X, O, H)
#obj_old,infoK_Old, infoS_Old=OR_BatterySize_old(X, O, H)


#Slack på constraint 3_1 betyder, at du kan køre 138 km mere
#Slack på constraint 5_1 betyder, at du kan oplade 600 km mere 

In [15]:
# Create a battery charging plot based on OR model output
def Battery_Charging_plot(infoK, infoS):
    infoS=pd.DataFrame(infoS)
    infoK=pd.DataFrame(infoK)
    
    days=['Monday','Tuesday','Wednesday',"Thursday", "Friday", "Saturdaty", "Sunday"]*5
    #days=range(0,30)
    #days.insert(0,"Start")
    #days.insert(0,0)
    fig, ax=plt.subplots(figsize=(20,6))
    ax.plot(range(1,30),infoS['value'], 'o-',label="Battery Status")
    ax.plot(range(1,30),infoK['value'], 'o-',label="Charging", color='green')
    ax.legend()
    ax.set(xticks=range(1,30),xticklabels=days[:29]);
    ax.set_xticklabels(labels=days[:29],rotation=45);
    ax.set_ylabel('Kilometers',fontsize=15);
    ax.set_xlabel('Weekday of a Month',fontsize=15)
    ax.set_title('Your Battery Status and Charging Pattern', fontsize=15);
    #plt.savefig('Battery_Charge_Status1.eps', format='eps')
    plt.close()
    
    return fig


In [16]:
# Function that takes in the user's input an convert it to the right format before calculating the optimal battery range
def User_Preferences(days, when, km, r, lat, lon):
    # Create availability dataframe for every hour of the week based on predictions from each charger
    df_availability=1-(df_predictions.groupby((df_predictions.index.dayofweek) * 24 +  (df_predictions.index.hour)).mean())
    df_availability.columns=[str(int(i)) for i in df_availability.columns.tolist()]

    #Get all CPIDs near you
    CPIDs=find_chargers_near_you(r,lat,lon,df_all)

    #Check if there are any CPIDs if not return function
    if len(CPIDs)>0:
       # Create dict of type of chargers near you 
        temp_dict=dict(df_all[df_all['CP ID'].isin(CPIDs)].groupby(['Type'])['CP ID'].nunique())
        # Find the missing types
        missing=[i for i in ['Fast','Slow','Rapid','Unknown'] if i not in list(temp_dict.keys())]
        # Update the dictionary
        temp_dict.update({k:0 for k in missing})
        # Put into an array:
        charge_type=np.array([temp_dict['Rapid'],temp_dict['Fast'],temp_dict['Slow'],temp_dict['Unknown']])
        #Carlculate expected charge speed given chargers in area(rapid, fast, slow, unknown)
        H=np.sum([i for i in charge_type*[120,60,20,60] if i!=0])/np.sum(charge_type)


    else:
        return False, False, False, print('No available EV Chargers in your area. Try increasing the meters you are willing to walk or try again later!')


    # Calculate all day hours and all night hours
    nat=[[i+day*24 for i in range(0,8)] for day in range(7)]
    day=[[i+day*24 for i in range(8,20)] for day in range(7)]
    nat2=[[i+day*24 for i in range(20,24)] for day in range(7)]
    #Extend nat
    [nat[i].extend(nat2[i]) for i in range(len(nat))]


    # Check if the user have chosen days to charge if not return else get index of those days
    if len(days)==0:
        return False, False, False, print('Choose the days you wish to charge')        

    else:
        weekdays=['Monday','Tuesday','Wednesday', 'Thursday','Friday','Saturday','Sunday']
        index_days=[num for num,i in enumerate(weekdays) if i in days]


    # User specified data: Night
    if when=='Night':
        # Get only nights of chosen weekdays
        nat_flat=[i for num,i in enumerate(nat) if num in index_days]
        nat_flat=[item for sublist in nat_flat for item in sublist]

        # Get index of all hours with availability above 0.45
        df_temp=pd.DataFrame(((df_availability[CPIDs].mean(axis=1).loc[nat_flat]>0.45)*1))
        index=df_temp.loc[df_temp[0]>0].index.tolist()

        # Count number of available hours for every weekday given preference
        O=[len([i for i in index if i in j ]) for j in nat]*5
        O.insert(0,0)
        km.insert(0,0)


        try:
            Batterysize,infoK, infoS=OR_BatterySize(km, O, H)
            car_index=list(EVs['Range']).index(min(i for i in EVs['Range'] if i>Batterysize))
            
            return infoK, infoS, index, print('With your needs a',EVs.iloc[car_index]['Model'], "is sufficient, since you need a car with a range of",Batterysize,"kilometers")

        except:
            return False, False, False, print('No available EVs for your needs yet.. Try again later!')

    if when=='Day':
        # Get only days of chosen weekdays
        day_flat=[i for num,i in enumerate(day) if num in index_days]
        day_flat=[item for sublist in day_flat for item in sublist]

        # Get index of all hours with availability above 0.45
        df_temp=pd.DataFrame(((df_availability[CPIDs].mean(axis=1).loc[day_flat]>0.45)*1))
        index=df_temp.loc[df_temp[0]>0].index.tolist()


        # Count number of available hours for every weekday given preference
        O=[len([i for i in index if i in j ]) for j in day]*5
        O.insert(0,0)
        km.insert(0,0)

        try:
            Batterysize,infoK, infoS=OR_BatterySize(km, O, H)
            car_index=list(EVs['Range']).index(min(i for i in EVs['Range'] if i>Batterysize))
            return infoK, infoS, index, print('With your needs a',EVs.iloc[car_index]['Model'], "is sufficient, since you need a car with a range of",Batterysize,"kilometers")

        except:
            return False, False, False, print('No available EVs for your needs yet.. Try again later!')

In [17]:
#input:
#days=['Monday', 'Tuesday', 'Friday', 'Sunday',]
#when='Night'
#km=[60]*7*5
#r=1
#lat = 56.4608750723589
#lon = -2.96901366370135
#info=User_Preferences(days, when, km, r, lat, lon)

## Choose your preferences

In [18]:
# Packages for interface
from IPython.display import display
import ipywidgets as widgets
from ipywidgets import interact, IntSlider, interactive, widgets, interact_manual,HBox,fixed, Layout, GridBox, VBox, HBox
from geopy.geocoders import Nominatim

In [19]:
# Create the widgets
caption0 = widgets.Label(value='Please enter your Address:')

#### Latitude and Longitude
Streetname=widgets.Text(disabled=False, description='Street') #Castle street 3
City=widgets.Text(disabled=False,description='City', value='Dundee' )

caption1 = widgets.Label(value='Please choose how many meters you are willing to walk to reach a charger:')
#### Kilometers
Radius=widgets.IntSlider(
    #value=600,
    min=0,
    max=2000,
    step=50,
    description='0',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    layout = Layout(width="70%"),
    readout_format='d'
)



caption2 = widgets.Label(value='Please enter the average kilometers you drive on a weekly basis:')
#### Kilometers
Kilometers=widgets.IntSlider(
    #value=150,
    min=0,
    max=1000,
    step=10,
    description='0',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    layout = Layout(width="70%"),
    readout_format='d'
)

Km_Mon=widgets.BoundedIntText(
    min=0,
    max=500,
    step=1,
    description='Monday:',
    layout = Layout(width="50%"),
    disabled=False
)

Km_Tue=widgets.BoundedIntText(
    min=0,
    max=500,
    step=1,
    description='Tuesday:',
    layout = Layout(width="50%"),
    disabled=False
)
Km_Wed=widgets.BoundedIntText(
    min=0,
    max=500,
    step=1,
    description='Wednesday:',
    layout = Layout(width="50%"),
    disabled=False
)
Km_Thur=widgets.BoundedIntText(
    min=0,
    max=500,
    step=1,
    description='Thursday:',
    layout = Layout(width="50%"),
    disabled=False
)

Km_Fri=widgets.BoundedIntText(
    min=0,
    max=500,
    step=1,
    description='Friday:',
    layout = Layout(width="50%"),
    disabled=False
)
Km_Sat=widgets.BoundedIntText(
    min=0,
    max=500,
    step=1,
    description='Saturday:',
    layout = Layout(width="50%"),
    disabled=False
)
Km_Sun=widgets.BoundedIntText(
    min=0,
    max=500,
    step=1,
    description='Sunday:',
    disabled=False,
    layout = Layout(width="50%")
)

#### Weekdays

caption3 = widgets.Label(value='Please check the weekdays you wish to charge:')
Mon = widgets.Checkbox(
    indent=False,
    value=False,
    description='Monday',
    layout = Layout(width="50%"),
    disabled=False,
)

Tues = widgets.Checkbox(
    indent=False,
    value=False,
    description='Tuesday',
    layout = Layout(width="50%"),
    disabled=False,
)
Wednes= widgets.Checkbox(
    indent=False,
    value=False,
    description='Wednesday',
    layout = Layout(width="50%"),
    disabled=False,
)
Thurs = widgets.Checkbox(
    indent=False,
    value=False,
    description='Thursday',
    layout = Layout(width="50%"),
    disabled=False,
)
Fri = widgets.Checkbox(
    indent=False,
    value=False,
    description='Friday',
    layout = Layout(width="50%"),
    disabled=False,
)
Satur = widgets.Checkbox(
    indent=False,
    value=False,
    description='Saturday',
    layout = Layout(width="50%"),
    disabled=False,
)
Sun= widgets.Checkbox(
    indent=False,
    value=False,
    description='Sunday',
    layout = Layout(width="50%"),
    disabled=False,
)




#### Days a week
caption4 = widgets.Label(value='Please choose if you want to charge day or night:')
Choose_time_a_day = widgets.RadioButtons(
    indent=False,
    options=['Day','Night'],
    layout = Layout(width="50%"),
    description=' ',
    disabled=False,
)





button = widgets.Button(description='Click to calculate result',
           layout=Layout(width='50%', height='80px'))
output = widgets.Output()


def on_button_clicked(b):
    with output:
        #plots=run_script(Streetname, City,Radius, Kilometers, Choose_time_a_day,Mon, Tues, Wednes,Thurs,Fri,Satur,Sun)
        #print(plots)
        try:
            output.clear_output()
            plots=run_script(Streetname, City,Radius, Kilometers, Choose_time_a_day,Mon, Tues, Wednes,Thurs,Fri,Satur,Sun)
            if plots[0]!=False:
                print('\nSee your monthly battery status and charging pattern in the graph below.')
                display(plots[0])
                print('\nExplore the availability in your area below. Click on the black dots and the blue pin to see more.')
                display(plots[1])
                # print number of available chargers in grid
                print(f'There are {plots[2]} EV charger(s) in you area using a radius of {Radius.value} m')
        except:
            print('Invalid information - try again')

button.on_click(on_button_clicked)



#Define groups of widgets
First=widgets.VBox([caption0,Streetname,City])
First2=widgets.VBox([caption1,Radius])
Second=widgets.VBox([caption2,Kilometers])
#temp=widgets.HBox([Km_Mon,Km_Tue,Km_Wed,Km_Thur,Km_Fri,Km_Sat,Km_Sun])
#Second=widgets.VBox([caption2,temp])
temp=widgets.HBox([Mon,Tues,Wednes,Thurs,Fri,Satur,Sun])
Third=widgets.VBox([caption3,temp])
Fourth=widgets.VBox([caption4,Choose_time_a_day])


# Display with newline inbetween
display(First)
print("\n")
display(First2)
print("\n")
display(Second)
print("\n")
display(Third)
print("\n")
display(Fourth)
print("\n")
widgets.VBox([button, output])

#Lothian Crescent 300

In [20]:
# Function that ensures everything is evaluated when the user hits the gray button
def run_script(Streetname, City,Radius, Kilometers, Choose_time_a_day,Mon, Tues, Wednes,Thurs,Fri,Satur,Sun):
    
    #get the coordinates from adress
    geolocator = Nominatim(timeout=1000, user_agent='myGeocoder')
    address = Streetname.value +", "+City.value
    geo=geolocator.geocode(address,timeout=1000)
    
    lat=geo.latitude
    lon=geo.longitude

    #Get radius: 
    r=int(Radius.value)/1000
    
    # Get km
    km=[round((Kilometers.value)/7)]*7*5
    
    #Get When 
    when=Choose_time_a_day.value
    
    #Get days
    weekdays=['Monday','Tuesday','Wednesday', 'Thursday','Friday','Saturday','Sunday']
    binary_days=np.array([Mon.value,Tues.value,Wednes.value,Thurs.value,Fri.value, Satur.value, Sun.value])*1
    days=[weekdays[i] for i in range(len(binary_days)) if binary_days[i]!=0]
    
    if Streetname.value=='':
        return False, print('Type in your street name and number')
    
    if len(days)==0:
         return False, print('Choose the days you wish to charge')
        
    if r==0:
         return False, print('Choose how far you are willing to walk')
        

    else:
        # print car status
        info=User_Preferences(days, when, km, r, lat, lon)
        num_chargers=len(find_chargers_near_you(r,lat,lon,df_all))
        
        if info[0]!=False:
            return [Battery_Charging_plot(info[0],info[1]),heatmap(df_predictions,lat,lon,r,info[2]), num_chargers]
        
        else:
            if info[2]!=False:
                return [False,heatmap(df_predictions,lat,lon,r, info[2]), num_chargers]
            else:
                return[False,False,False]
        

